In [1]:
# Install Dependencies
import pandas as pd
import scipy.stats as stat
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Set Data Paths
mouse_metadata_path = '../data/Mouse_metadata.csv'
study_results_path = '../data/study_results.csv'

# Load Data Into Pandas
mouse_metadata_df = pd.read_csv(mouse_metadata_path)
study_results_df = pd.read_csv(study_results_path)

study_data_df = pd.merge(study_results_df, mouse_metadata_df, how='left', on='Mouse ID')

study_data_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [2]:
# Get duplicate values from study data
duplicate_mouse_row = study_data_df.duplicated(subset=['Mouse ID', 'Timepoint'])
duplicate_mouse_df = study_data_df[duplicate_mouse_row]
duplicate_mouse_id_list = duplicate_mouse_df['Mouse ID'].unique()

# Drop all mice that have duplicate study rows
study_data_df = study_data_df[~study_data_df['Mouse ID'].isin(duplicate_mouse_id_list)]